In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccmmrb27',
    proc_date,
    'temp_sap_vccmmrb27'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb27 (
		kappl STRING, 
    kschl STRING, 
    vkorg STRING, 
    vtweg STRING, 
    werks STRING, 
    matnr STRING, 
    vrkme STRING, 
    datbi DATE, 
    recordmode STRING, 
    datab DATE, 
    knumh STRING, 
    proc_date DATE,
    file_creation_ts TIMESTAMP
        )
    TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_vccmmrb27
AS
SELECT
  kappl,
  kschl,
  vkorg,
  vtweg,
  werks,
  LTRIM( '0',matnr) matnr,
  vrkme,
  TO_DATE(datbi,'yyyyMMdd') datbi,
  recordmode,
  TO_DATE(datab, 'yyyyMMdd') datab,
  knumh,
  proc_date,
  file_creation_ts
FROM temp_sap_vccmmrb27;
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb27
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb27
SELECT * FROM sap_vccmmrb27
""")